In [1]:
import os
import pandas as pd
from qiime2 import Visualization
import matplotlib.pyplot as plt
import numpy as np

import qiime2 as q2

%matplotlib inline


In [2]:
data_dir = 'Alien_data'

if not os.path.isdir(data_dir):
    os.makedirs(data_dir)

In [8]:
! wget -nv -O $data_dir/sequences.qza 'https://polybox.ethz.ch/index.php/s/PCQspFMocVCKjZ3/download'

2022-10-17 17:38:52 URL:https://polybox.ethz.ch/index.php/s/PCQspFMocVCKjZ3/download [3433846903/3433846903] -> "Alien_data/sequences.qza" [1]


In [56]:
! wget -nv -O $data_dir/sample_metadata.tsv 'https://polybox.ethz.ch/index.php/s/r1AYzdUVWnQyiRL/download'

2022-10-10 19:45:54 URL:https://polybox.ethz.ch/index.php/s/r1AYzdUVWnQyiRL/download [10012/10012] -> "Alien_data/sample_metadata.tsv" [1]


In [59]:
metadata_df = pd.read_csv(f'{data_dir}/sample_metadata.tsv', sep='\t')

In [3]:
! qiime tools peek $data_dir/sequences.qza

UUID:        394c4773-80e2-46a6-9fba-40e7c8ec3fb9
Type:        SampleData[PairedEndSequencesWithQuality]
Data format: SingleLanePerSamplePairedEndFastqDirFmt


In [10]:
! qiime demux summarize \
    --i-data $data_dir/sequences.qza \
    --o-visualization $data_dir/sequences.qzv

Saved Visualization to: Alien_data/sequences.qzv


In [8]:
Visualization.load(f'{data_dir}/sequences.qzv')

<visualization: Visualization uuid: 3e2d23ea-b49d-454f-9b2b-3a4288c37e57>

Cutadapt is used to trime out the forward and reverse primers:

In [4]:
! qiime cutadapt trim-paired \
  --i-demultiplexed-sequences $data_dir/sequences.qza \
  --p-front-f AYTGGGYDTAAAGNG \
  --p-front-r CCGTCAATTYHTTTRAGT \
  --p-error-rate 0 \
  --o-trimmed-sequences $data_dir/primer-trimmed-seqs.qza \
  --verbose

Running external command line application. This may print messages to stdout and/or stderr.
The commands to be run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt --cores 1 --error-rate 0 --times 1 --overlap 3 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-p4d5rt0g/0DOSLC_113_L001_R1_001.fastq.gz -p /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-p4d5rt0g/0DOSLC_274_L001_R2_001.fastq.gz --front AYTGGGYDTAAAGNG -G CCGTCAATTYHTTTRAGT /tmp/qiime2-archive-we88jvpb/394c4773-80e2-46a6-9fba-40e7c8ec3fb9/data/0DOSLC_113_L001_R1_001.fastq.gz /tmp/qiime2-archive-we88jvpb/394c4773-80e2-46a6-9fba-40e7c8ec3fb9/data/0DOSLC_274_L001_R2_001.fastq.gz

This is cutadapt 4.0 with Python 3.8.13
Command line parameters: --cores 1 --error-rate 0 --times 1 --overlap 3 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-p4d5rt0g/0DOSLC_113_L001_R1_001.fastq.gz -p /tmp/q2-CasavaOneEightSi

In [6]:
! qiime demux summarize \
  --i-data $data_dir/primer-trimmed-seqs.qza \
  --o-visualization $data_dir/primer-trimmed-seqs.qzv

Saved Visualization to: Alien_data/primer-trimmed-seqs.qzv


In [15]:
Visualization.load(f'{data_dir}/primer-trimmed-seqs.qzv')

<visualization: Visualization uuid: a3e8ac5e-8ddf-4645-aca7-bc0bd3d885ca>

Dada 2 is used to denoise the paired-end sequences. The truncation length is selected based on median value of quality score larger than 30. 

In [8]:
! qiime dada2 denoise-paired \
    --i-demultiplexed-seqs $data_dir/primer-trimmed-seqs.qza \
    --p-trunc-len-f 183 \
    --p-trunc-len-r 190 \
    --p-n-threads 4 \
    --o-table $data_dir/dada2_table.qza \
    --o-representative-sequences $data_dir/dada2_rep_set.qza \
    --o-denoising-stats $data_dir/dada2_stats.qza

Saved FeatureTable[Frequency] to: Alien_data/dada2_table.qza
Saved FeatureData[Sequence] to: Alien_data/dada2_rep_set.qza
Saved SampleData[DADA2Stats] to: Alien_data/dada2_stats.qza


In [9]:
! qiime metadata tabulate \
    --m-input-file $data_dir/dada2_stats.qza \
    --o-visualization $data_dir/dada2_stats.qzv

Saved Visualization to: Alien_data/dada2_stats.qzv


In [16]:
Visualization.load(f'{data_dir}/dada2_stats.qzv')

<visualization: Visualization uuid: 8f87926c-2fde-4497-bfe3-3c6ea402adfd>

In [11]:
! qiime feature-table summarize \
  --i-table $data_dir/dada2_table.qza \
  --o-visualization $data_dir/dada2_table.qzv

Saved Visualization to: Alien_data/dada2_table.qzv


In [12]:
Visualization.load(f'{data_dir}/dada2_table.qzv')

<visualization: Visualization uuid: 2a7f7529-81a8-47d3-a192-75249483c000>

In [13]:
! qiime feature-table tabulate-seqs \
  --i-data $data_dir/dada2_rep_set.qza \
  --o-visualization $data_dir/dada2_rep_set.qzv

Saved Visualization to: Alien_data/dada2_rep_set.qzv


In [14]:
Visualization.load(f'{data_dir}/dada2_rep_set.qzv')

<visualization: Visualization uuid: 92e1a1db-1b09-4bf5-869e-d75c9035d036>

In [ ]:
Taxonomy classification:

Database curation:

In [15]:
! qiime rescript get-silva-data \
    --p-version '138.1' \
    --p-target 'SSURef_NR99' \
    --p-include-species-labels \
    --o-silva-sequences $data_dir/silva-138.1-ssu-nr99-rna-seqs.qza \
    --o-silva-taxonomy $data_dir/silva-138.1-ssu-nr99-tax.qza

Saved FeatureData[RNASequence] to: Alien_data/silva-138.1-ssu-nr99-rna-seqs.qza
Saved FeatureData[Taxonomy] to: Alien_data/silva-138.1-ssu-nr99-tax.qza


In [16]:
 ! qiime rescript cull-seqs \
     --i-sequences $data_dir/silva-138.1-ssu-nr99-rna-seqs.qza \
     --p-num-degenerates 5 \
     --p-homopolymer-length 8 \
     --p-n-jobs 3 \
     --o-clean-sequences $data_dir/silva-138.1-ssu-nr99-rna-seqs-cleaned.qza

Saved FeatureData[Sequence] to: Alien_data/silva-138.1-ssu-nr99-rna-seqs-cleaned.qza


In [17]:
! qiime rescript filter-seqs-length-by-taxon \
    --i-sequences $data_dir/silva-138.1-ssu-nr99-rna-seqs-cleaned.qza \
    --i-taxonomy $data_dir/silva-138.1-ssu-nr99-tax.qza \
    --p-labels Archaea Bacteria Eukaryota \
    --p-min-lens 900 1200 1400 \
    --o-filtered-seqs $data_dir/silva-138-ssu-nr99-seqs-filt.qza \
    --o-discarded-seqs $data_dir/silva-138-ssu-nr99-seqs-discard.qza

Saved FeatureData[Sequence] to: Alien_data/silva-138-ssu-nr99-seqs-filt.qza
Saved FeatureData[Sequence] to: Alien_data/silva-138-ssu-nr99-seqs-discard.qza


In [4]:
! qiime rescript dereplicate \
    --i-sequences $data_dir/silva-138-ssu-nr99-seqs-filt.qza  \
    --i-taxa $data_dir/silva-138.1-ssu-nr99-tax.qza \
    --p-rank-handles 'silva' \
    --p-mode 'uniq' \
    --p-threads 3 \
    --o-dereplicated-sequences $data_dir/silva-138-ssu-nr99-seqs-derep-uniq.qza \
    --o-dereplicated-taxa $data_dir/silva-138-ssu-nr99-tax-derep-uniq.qza

Saved FeatureData[Sequence] to: Alien_data/silva-138-ssu-nr99-seqs-derep-uniq.qza
Saved FeatureData[Taxonomy] to: Alien_data/silva-138-ssu-nr99-tax-derep-uniq.qza


PCR extraction: 
The primers used for our samples are designed for the V4-V5 region:
563 F: 5′-AYTGGGYDTAAAGNG-3′
926 R: 5′-CCGTCAATTYHTTTRAGT-3′

In [15]:
! qiime feature-classifier extract-reads \
    --i-sequences $data_dir/silva-138-ssu-nr99-seqs-derep-uniq.qza \
    --p-f-primer AYTGGGYDTAAAGNG \
    --p-r-primer CCGTCAATTYHTTTRAGT \
    --p-n-jobs 3 \
    --p-read-orientation 'forward' \
    --o-reads $data_dir/silva-138-ssu-nr99-seqs-515f-806r.qza

Saved FeatureData[Sequence] to: Alien_data/silva-138-ssu-nr99-seqs-515f-806r.qza


In [16]:
! qiime rescript dereplicate \
    --i-sequences $data_dir/silva-138-ssu-nr99-seqs-515f-806r.qza \
    --i-taxa $data_dir/silva-138-ssu-nr99-tax-derep-uniq.qza \
    --p-rank-handles 'silva' \
    --p-mode 'uniq' \
    --p-threads 3 \
    --o-dereplicated-sequences $data_dir/silva-138-ssu-nr99-seqs-515f-806r-uniq.qza \
    --o-dereplicated-taxa  $data_dir/silva-138-ssu-nr99-tax-515f-806r-derep-uniq.qza

Saved FeatureData[Sequence] to: Alien_data/silva-138-ssu-nr99-seqs-515f-806r-uniq.qza
Saved FeatureData[Taxonomy] to: Alien_data/silva-138-ssu-nr99-tax-515f-806r-derep-uniq.qza


We tried to use the silva database and train the classifers using naive bayes, but it exceeded the memory capacity to train the classifer and also to use the pre-trained Silva database, so we decided to use the pre-trained Greengene classifier. 

In [17]:
#! qiime feature-classifier fit-classifier-naive-bayes \
#     --i-reference-reads $data_dir/silva-138-ssu-nr99-seqs-515f-806r-uniq.qza \
#     --i-reference-taxonomy $data_dir/silva-138-ssu-nr99-tax-515f-806r-derep-uniq.qza \
#     --p-classify--chunk-size 1000 \
#     --o-classifier $data_dir/515f-806r-classifier.qza

In [17]:
! wget -nv -O $data_dir/gg-13-8-99-nb-classifier.qza 'https://data.qiime2.org/2022.8/common/gg-13-8-99-nb-classifier.qza'

2022-10-26 19:41:49 URL:https://s3-us-west-2.amazonaws.com/qiime2-data/2022.8/common/gg-13-8-99-nb-classifier.qza [104512483/104512483] -> "Alien_data/gg-13-8-99-nb-classifier.qza" [1]


In [18]:
! qiime feature-classifier classify-sklearn \
    --i-classifier $data_dir/gg-13-8-99-nb-classifier.qza \
    --i-reads $data_dir/dada2_rep_set.qza \
    --o-classification $data_dir/taxonomy.qza

Saved FeatureData[Taxonomy] to: Alien_data/taxonomy.qza


In [19]:
! qiime metadata tabulate \
    --m-input-file $data_dir/taxonomy.qza \
    --o-visualization $data_dir/taxonomy.qzv

Saved Visualization to: Alien_data/taxonomy.qzv


In [20]:
Visualization.load(f'{data_dir}/taxonomy.qzv')

<visualization: Visualization uuid: dd568eab-270d-4892-9f13-ae4bb3dc3822>

In [5]:
! qiime taxa barplot \
    --i-table $data_dir/dada2_table.qza \
    --i-taxonomy $data_dir/taxonomy.qza \
    --m-metadata-file $data_dir/sample_metadata.tsv \
    --o-visualization $data_dir/taxa-bar-plots.qzv

Saved Visualization to: Alien_data/taxa-bar-plots.qzv


In [6]:
Visualization.load(f'{data_dir}/taxa-bar-plots.qzv')

<visualization: Visualization uuid: 0f39b3b8-b8f7-4d8f-84e3-f77b18f1d391>

Mitochondira and chloroplast are filtered since they don't belong to the gut microbiota:

In [7]:
! qiime taxa filter-table \
    --i-table $data_dir/dada2_table.qza \
    --i-taxonomy $data_dir/taxonomy.qza \
    --p-exclude mitochondria,chloroplast \
    --o-filtered-table $data_dir/table-filtered.qza

Saved FeatureTable[Frequency] to: Alien_data/table-filtered.qza


In [9]:
! qiime taxa filter-seqs \
    --i-sequences $data_dir/dada2_rep_set.qza \
    --i-taxonomy $data_dir/taxonomy.qza \
    --p-exclude mitochondria,chloroplast \
    --o-filtered-sequences $data_dir/rep-seqs-filtered.qza

Saved FeatureData[Sequence] to: Alien_data/rep-seqs-filtered.qza


In [13]:
! qiime taxa barplot \
    --i-table $data_dir/table-filtered.qza \
    --i-taxonomy $data_dir/taxonomy.qza \
    --m-metadata-file $data_dir/sample_metadata.tsv \
    --o-visualization $data_dir/taxa-bar-plots_filtered.qzv

Saved Visualization to: Alien_data/taxa-bar-plots_filtered.qzv


In [14]:
Visualization.load(f'{data_dir}/taxa-bar-plots_filtered.qzv')

<visualization: Visualization uuid: 6bf8bdcd-fb82-4933-b20f-aa69ef72167c>